In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn.metrics
import warnings
warnings.filterwarnings("ignore")

/home/shikha/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
data = pd.read_csv('../notebooks/clean_df.csv')
data.head()

,Unnamed: 0,Sample No.,Patient ID,Study ID,Age,Gender,BMI,Symptoms Presented,Symptoms Present?,Fever,...,ALKALINE PHOSPHATASE,C-REACTIVE PROTEINS,ECG,CXR,CORADS SCORE ON HRCT,PATIENT CATEGORY,PLASMA THERAPY,Age Above 60?,Temp Groups,CORADS 6
0,0,1013,IGIB1040201013,IGIB1040201013-1,0.551724,1,0.436782,"fever, cough,breathlessness",1,1,...,0.127089,0.152271,sinus rhythm,normal,4.0,5a,No,0,0.2,0.0
1,1,1014,IGIB1040201014,IGIB1040201014-1,0.241379,0,0.804598,asymptomatic,0,0,...,0.172501,0.009346,sinus rhythm,normal,0.0,5b,No,0,0.6,0.0
2,2,1015,IGIB1040201015,IGIB1040201015-1,0.264368,1,0.402299,asymptomatic,0,0,...,0.259224,0.026437,sinus rhythm,normal,0.0,5b,No,0,0.4,0.0
3,3,1016,IGIB1040201016,IGIB1040201016-1,0.781609,1,0.321839,"fever, cough,breathlessness",1,1,...,0.366446,0.443161,sinus rhythm,normal,5.0,4,No,1,0.4,0.0
4,4,1060,IGIB1040201060,IGIB1040201060-1,0.505747,1,1.000000,"fever,cough",1,1,...,0.546200,0.430297,sinus rhythm,normal,5.0,4,No,0,0.8,0.0


## Model

In [3]:
def model_perf(x,y,model):
    cm = metrics.confusion_matrix(y, model.predict(x))
    plt.figure
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    plt.savefig('covidseverity.png')
    accuracy = model.score(x,y)
    precision,recall,f1,support = sklearn.metrics.precision_recall_fscore_support(y, model.predict(x),average='binary')
    print("Accuracy Score = ",accuracy)
    print("Precision Score = ",precision)    
    print("Recall Score = ",recall)    
    print("F1 Score = ",f1)

In [4]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [5]:
def visualize(x,y, model):
    
        cm = metrics.confusion_matrix(y, model.predict(x))
        plt.figure
        sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
        plt.ylabel('Actual label');
        plt.xlabel('Predicted label');
        plt.savefig('covidseverity.png')
        
        
def ttmodel(X, Y, lm, n):
    
        train_accuracy = [0]*n
        train_precision = [0]*n
        train_recall = [0]*n
        train_f1 = [0]*n

        test_accuracy = [0]*n
        test_precision = [0]*n
        test_recall = [0]*n
        test_f1 = [0]*n
        
        for i in range(n):
            x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

            lm.fit(x_train,y_train)

            train_accuracy[i] = lm.score(x_train,y_train)
            train_precision[i], train_recall[i], train_f1[i], s = sklearn.metrics.precision_recall_fscore_support(y_train, lm.predict(x_train), average='weighted')

            test_accuracy[i] = lm.score(x_test,y_test)
            test_precision[i], test_recall[i], test_f1[i], s = sklearn.metrics.precision_recall_fscore_support(y_test, lm.predict(x_test), average='weighted')

        print("Average training statistics over",n,"runs of the NN model - ")
        print("Accuracy Score = ", sum(train_accuracy)/n)
        print("Precision Score = ", sum(train_precision)/n)    
        print("Recall Score = ", sum(train_recall)/n)   
        print("F1 Score = ", sum(train_f1)/n) 

        print()
        
        print("Average testing statistics over",n,"runs of the NN model - ")
        print("Accuracy Score = ", sum(test_accuracy)/n)
        print("Precision Score = ", sum(test_precision)/n)    
        print("Recall Score = ", sum(test_recall)/n)   
        print("F1 Score = ", sum(test_f1)/n) 
        
        visualize(x_test, y_test, lm)
        
        
def cvmodel(X, Y, lm, cv):
    
        from sklearn.model_selection import cross_validate
        scoring = ['f1','roc_auc']
        scores = cross_validate(lm, X, Y, cv=cv, scoring = scoring, return_train_score=True)
        
        for k,v in scores.items():
            ans = [min(v),sum(v)/len(v),max(v)]
            print(k,"=",ans)
            print()

## Initial Factors + Symptoms and Comorbidities + Blood

In [6]:
X = data[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)','Symptoms Present?','Co-morbidity?','Cough','Breathlessness','Hypertension','C-REACTIVE PROTEINS','UREA']]
Y = data['Outcome']
trainset =  pd.read_csv('../notebooks/train.csv')
testset =  pd.read_csv('../notebooks/test.csv')

Xtrain = trainset[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)','Symptoms Present?','Co-morbidity?','Cough','Breathlessness','Hypertension','C-REACTIVE PROTEINS','UREA']]
ytrain = trainset['Outcome']

Xtest = testset[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)','Symptoms Present?','Co-morbidity?','Cough','Breathlessness','Hypertension','C-REACTIVE PROTEINS','UREA']]
ytest = testset['Outcome']

In [7]:
model = MLPClassifier()
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,100)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['lbfgs','sgd'],
    'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
    'learning_rate': ['constant','adaptive','invscaling'],
}

model, pred = algorithm_pipeline(Xtrain, Xtest, ytrain, ytest, model, param_grid, 
                                 cv=5, scoring_fit ='f1')

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.0s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50,), learning_rate

[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.6s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.6s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   1.0s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.9s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.4s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.3s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=

[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=lbfgs; total time=   0.3s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=lbfgs; total time=   0.3s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=tanh, alpha=0.0003, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.6s
[CV] END activatio

[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.1s
[CV] END activatio

[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=sgd; total time=   0.5s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=sgd; total time=   0.6s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=sgd; total time=   0.5s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=sgd; total time=   0.5s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=sgd; total time=   0.4s
[CV] END activation=tanh, alpha=0.003, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.003, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=tanh, alpha=0.003, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV]

...

[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.6s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=lbfgs; total time=   0.7s
[CV] END activation=logistic, alpha=0.01, hidden_layer_sizes=(50, 100), learning_rate=invscaling, solver=

[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=sgd; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=sgd; total time=   0.3s
[CV] END activation=logistic, alpha=0.03, hidden_layer_sizes=(100,), learning_rate=invscaling, solver=sgd; t

[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(50,), learning_rate=invscaling, solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=logistic, alpha=0.1, hidden_layer_sizes=(100,), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] 

[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activatio

[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.2s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.5s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.4s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.6s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=lbfgs; total time=   0.4s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; t

In [24]:
model = MLPClassifier()
param_grid = {
    'hidden_layer_sizes': [(50,100),(50,50)],
    'activation': ['relu','logistic'],
    'solver': ['lbfgs','sgd'],
    'alpha': [0.1, 0.3, 1, 3],
    'learning_rate': ['constant','adaptive'],
}

model, pred = algorithm_pipeline(Xtrain, Xtest, ytrain, ytest, model, param_grid, 
                                 cv=5, scoring_fit ='f1')

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.4s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.4s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.4s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(50, 100), learnin

[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=0.3, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] END activation=re

[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.4s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 100), learning_rate=adaptive, solver=sgd; total time=   0.3s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=3, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=lbfgs; total time=   0.2s
[CV] END activation=relu, alpha=3, h

[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=lbfgs; total time=   0.3s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=0.3, hidden_layer_sizes=(50, 100), learning_rate=constant, solver=sgd; total

[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs; total time=   0.0s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] E

In [29]:
model = MLPClassifier(hidden_layer_sizes = (50,100), activation = 'relu', learning_rate = 'adaptive', solver = 'lbfgs', alpha = 1)

In [30]:
cvmodel(X,Y,model,7)

fit_time = [0.317899227142334, 0.4092458997453962, 0.6249501705169678]

score_time = [0.007457256317138672, 0.007858991622924805, 0.008299589157104492]

test_f1 = [0.5, 0.7064625850340136, 0.8571428571428571]

train_f1 = [0.8421052631578948, 0.8892437136798038, 0.9523809523809523]

test_roc_auc = [0.8333333333333334, 0.9257884972170688, 1.0]

train_roc_auc = [0.98, 0.9874719392825705, 0.997046880767811]

